In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import numpy as np
from keras.models import Model
from keras.layers import Input, LSTM, GRU, RepeatVector, Embedding, Bidirectional, Activation, Dense, Dropout, TimeDistributed
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing import sequence
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import csv
import codecs
import h5py

Using TensorFlow backend.


In [2]:
def vectorize(X, numChars, maxLen):
    Z = np.zeros((maxLen, numChars), dtype=np.bool)
    for t, word in enumerate(X[:-1]):
        if word!=0:
            Z[t, word-1] = True
    return Z

In [3]:
def myGenerator(batch_size, numChars, maxLen):
    h5f = h5py.File('Data/mytestfile.hdf5','r')
    index=0
    # Create empty arrays to contain batch of arrays
    Z = np.zeros((batch_size, maxLen, numChars), dtype=np.bool)

    while True:
        for j in range(0, batch_size):
            Z[j,:,:] = vectorize(h5f['mydata'][index], numChars, maxLen)
            index+=1
            if index > 1011700:
                index=0
        yield (Z, Z)
    h5f.close()

In [4]:
def myValidationGenerator(batch_size, numChars, maxLen):
    h5f = h5py.File('Data/mytestfile.hdf5','r')
    index=1011712
    last=1020084
    # Create empty arrays to contain batch of arrays
    Z = np.zeros((batch_size, maxLen, numChars), dtype=np.bool)

    while True:
        for j in range(0, batch_size):
            Z[j,:,:] = vectorize(h5f['mydata'][index], numChars, maxLen)
            index+=1
            if index > last:
                index = 1011712
        yield (Z, Z)
    h5f.close()

In [5]:
import pickle
char_list, char_indices, indices_char = pickle.load(open( 'autoencoder_chars.pkl', "rb" ) )

In [22]:
import keras.backend as K
#create keras model
batch_size = 4096
nb_epoch = 200
steps_per_epoch = 245
val_steps_per_epoch = 6
maxLen = 200

latent_dim_out = 4
latent_dim_in = 8

timesteps = maxLen

inputs = Input(shape=(timesteps, len(char_list)))
print("inputs", K.int_shape(inputs) )

#(number of timesteps, number of features)
encoded1 = Bidirectional(LSTM(latent_dim_out, activation = 'relu', dropout = 0, return_sequences=False))(inputs) #make return_sequences=True when adding a second lstm
print("encoded1", K.int_shape(encoded1))

#encoded2 = LSTM(latent_dim_in)(encoded1)
#print("encoded2", K.int_shape(encoded2))

decoded = RepeatVector(timesteps)(encoded1)
print("decoded1", K.int_shape(decoded))

#decoded = LSTM(latent_dim_out, return_sequences=True)(decoded)
#print("decoded2", K.int_shape(decoded))

decoded = LSTM(len(char_list), activation = 'relu', dropout = .5, return_sequences=True)(decoded) #make return_sequences=True if anything is added after this lstm
print("decoded3", K.int_shape(decoded))

#decoded = Dropout(0.5)(decoded)
decoded = Dense(len(char_list)), input_shape=(200, 128)(decoded)
print("output", K.int_shape(decoded))
# is sigmoid the best choice?
#activation = Activation('relu')(decoded)

sequence_autoencoder = Model(inputs, decoded)
encoder = Model(inputs, encoded1)

#optimizer = Adam(lr = 0.005)
#optimizer = RMSprop(lr=0.01)
sequence_autoencoder.compile(loss='categorical_crossentropy', optimizer='adam')

early_stopping = EarlyStopping(patience=10, verbose=1)
checkpointer = ModelCheckpoint(filepath='autoencoder_weights.hdf5', verbose=1, save_best_only=True)
#model2.load_weights('autoencoder_weights.hdf5')


inputs (None, 200, 54)
encoded1 (None, 8)
decoded1 (None, 200, 8)
decoded3 (None, 200, 54)
output (None, 200, 54)


In [23]:
#train model on test data
sequence_autoencoder.fit_generator(myGenerator(batch_size, len(char_list), maxLen), 
          steps_per_epoch = steps_per_epoch, 
          epochs = nb_epoch,
          verbose = 1,
          validation_data = myValidationGenerator(batch_size, len(char_list), maxLen),
          validation_steps = val_steps_per_epoch,
          max_q_size = 3,
          workers = 1,
          callbacks=[early_stopping, checkpointer])

Epoch 1/200
245/245 [==============================] - 1563s - loss: 0.0289 - val_loss: 4.5422e-08
Epoch 2/200
 65/245 [======>.......................] - ETA: 1098s - loss: 4.5619e-08

KeyboardInterrupt: 

In [ ]:
sequence_autoencoder.save("autoencoder_model.hdf5")
#from keras.models import load_model
#sequence_autoencoder = load_model("seq2seq_model.h5")

In [ ]:
preds = sequence_autoencoder.predict(Z[:5,:,:], batch_size=64, verbose=0)

In [32]:
encoder.predict(next(Z), batch_size=1, verbose=0)

TypeError: Error when checking model : data should be a Numpy array, or list/dict of Numpy arrays. Found: (array([[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ..., 
        [False...

In [31]:
Z=myGenerator(2, len(char_list), maxLen)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

print("Training history")
fig = plt.figure(figsize=(10,4))
ax1 = fig.add_subplot(1, 2, 1)
plt.plot(sequence_autoencoder.history.history['loss'])
ax1.set_title('loss')
ax2 = fig.add_subplot(1, 2, 2)
plt.plot(sequence_autoencoder.history.history['val_loss'])
ax2.set_title('validation loss')

In [ ]:
h5f = h5py.File('Data/mytestfile.hdf5','r')
Z = h5f['mydata'][:10]
h5f.close()

In [ ]:
Z[0]

In [34]:
encoder.evaluate_generator(myValidationGenerator(3000, len(char_list), maxLen), steps = 1, max_q_size=1, workers=1)

RuntimeError: You must compile your model before using it.